# Expand dataset

In [2]:
import sys
import http.client
import requests
import json
import datetime
import statistics as stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random
import talib
import numpy as np

# Gym stuff
import gym
import gym_anytrading
from gym_anytrading.envs import StocksEnv

# Stable baselines - RL stuff
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C
from stable_baselines3 import DDPG
from stable_baselines3 import DQN
from stable_baselines3 import PPO

In [3]:
constituent_symbols = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'PEP', 'KO']

In [4]:
conn = http.client.HTTPSConnection("yh-finance.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "1ae05dcbfcmshf19fbd6d5f7d966p1aa148jsnb1eaf3cab955",
    'X-RapidAPI-Host': "yh-finance.p.rapidapi.com"
}

## Retrieve data from YH Finance API

In [5]:
indicators = {}
progress = 0
for symbol in constituent_symbols:
    print(f"Progress: {math.floor((progress*100)/len(constituent_symbols))}%")
    
    conn.request("GET", f"/stock/v2/get-chart?interval=1d&symbol={symbol}&range=%20&region=US&period1=1522195200&period2=1672531200", headers=headers)
    # conn.request("GET", f"/stock/v3/get-chart?interval=1d&symbol={symbol}&range=%20&region=US&period1=1522195200&period2=1672531200&includePrePost=true&useYfid=true&includeAdjustedClose=true&events=capitalGain%2Cdiv%2Csplit", headers=headers)
    res = conn.getresponse()
    data = res.read()

    result = json.loads(data.decode("utf-8"))["chart"]["result"][0]
    indicators[symbol] = {
        "date": list(map(lambda x: datetime.datetime.fromtimestamp(x).date(), result["timestamp"])),
        "high": result["indicators"]["quote"][0]["high"],
        "volume": result["indicators"]["quote"][0]["volume"],
        "open": result["indicators"]["quote"][0]["open"],
        "low": result["indicators"]["quote"][0]["low"],
        "close": result["indicators"]["quote"][0]["close"],
        "adj_close": result["indicators"]["adjclose"][0]["adjclose"]
    }
    
    progress += 1

Progress: 0%
Progress: 16%
Progress: 33%
Progress: 50%
Progress: 66%
Progress: 83%


In [6]:
dfs = {}
for symbol in constituent_symbols:
    dfs[symbol] = pd.DataFrame(indicators[symbol])

### Store data locally

In [7]:
progress = 0
for symbol in constituent_symbols:
    print(f"Progress: {math.floor((progress*100)/len(constituent_symbols))}%")
    
    f = open(f"extended_data/{symbol}.csv", "w")
    f.write(dfs[symbol].to_csv(index=False))
    f.close()

    progress += 1
print(f"Progress: {100}%")

Progress: 0%
Progress: 16%
Progress: 33%
Progress: 50%
Progress: 66%
Progress: 83%
Progress: 100%


## Retrieve data from local storage

In [ ]:
dfs = {}
progress = 0
for symbol in constituent_symbols:
    print(f"Progress: {math.floor((progress*100)/len(constituent_symbols))}%")

    dfs[symbol] = pd.read_csv(f"extended_data/stocks/{symbol}.csv")
    date = dfs[symbol]['date']
    dfs[symbol] = dfs[symbol].drop(['date'], axis = 1)
    dfs[symbol]['date'] = date.map(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

    progress += 1
print(f"Progress: {100}%")